In [19]:
import numpy as np
import pandas as pd
import datetime

from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [24]:
#-----------------READ DATA----------------
datafilename = 'DATA/MockDATA.csv'

Data = pd.read_csv(datafilename)[['cp_flag','date','exdate','delta','impl_volatility']] #try using more features later
Data['date'] = pd.to_datetime(Data['date'])
Data['exdate'] = pd.to_datetime(Data['exdate'])
# Data drop puts
Data = Data.drop(Data[Data['cp_flag'] == 'P'].index)

# sdate = datetime.datetime(2019,4,13)
# edate = datetime.datetime(2019,4,20)


# Data = Data[Data['date'] < edate] #change this line if we need more complicated set of data
# Data = Data[Data['date'] >= sdate]

Data['time_to_maturity'] = (Data['exdate']-Data['date']).astype('int64')/10**9/3600/24
Data = Data.dropna()

Data
Data = Data.dropna()

Data = Data.sort_values(by='date')
Data['impl_volatility'] = np.log(Data['impl_volatility'])

In [26]:
#Train Test Split

rate = 0.2
rows, columns = Data.shape
num = round(rows*(1-rate))

idx = Data.index
train = Data.loc[idx[0:num]]
test = Data.loc[idx[num:]]



X_train = train[['delta','time_to_maturity']]
X_test = test[['delta','time_to_maturity']]

y_train = train['impl_volatility']
y_test = test['impl_volatility']

In [27]:
bestneural = MLPRegressor(hidden_layer_sizes=(100,50,10),
                                          max_iter= 600, activation = 'tanh',
                                          solver='adam', learning_rate= 'constant')


bestneural.fit(X_train.values, y_train.values)

y_p_neur = bestneural.predict(X_test.values)

metrics.mean_squared_error(y_p_neur,y_test)

0.08816669325587453

In [28]:
from matplotlib import interactive

import matplotlib.pyplot as plt
from matplotlib import cm
#from mpl_toolkits.mplot3d import Axes3D
#Plot the fitting result
interactive(True)
#delta = 0.1, 0., 15, 0.2, 0.25, 0.3, 0.35, . . . , 0.9
#time to expiry of 1, 2, 3, 4, 8, 12, 18, 52, 104 weeks. 
D = np.linspace(-1, 1, 41, endpoint=True)
T = np.array([1,2,3,4,8,12,18,52,104,156])
T_days = T*7 #over time_to_maturity is in days

dd,tt = np.meshgrid(D,T_days)


X_plot = np.array([dd.flatten(), tt.flatten()]).T

y_plot = bestneural.predict(X_plot)

z = y_plot.reshape(dd.shape)

In [29]:

xd_test = np.array(X_test['delta'])
xt_test = np.array(X_test['time_to_maturity'])

test_z = np.array(y_test)


xd_train = np.array(X_train['delta'])
xt_train = np.array(X_train['time_to_maturity'])

train_z = np.array(y_train)

In [30]:
%matplotlib qt 
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(dd,tt, z,cmap =cm.coolwarm)


ax.scatter(xd_train, xt_train, train_z, color='green')
ax.scatter(xd_test, xt_test, test_z, color='red')

ax.set_xlabel('delta')
ax.set_ylabel('time to maturity')
ax.set_zlabel('Volatility');

plt.show()